In [13]:
import random
import math
from typing import List, Tuple, Optional

def distance(city1: Tuple[float, float], city2: Tuple[float, float]) -> float:
    # corrected: use squared difference (**2) not *2
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

def initialize_pheromone(num_cities: int, tau0: float) -> List[List[float]]:
    return [[tau0 for _ in range(num_cities)] for _ in range(num_cities)]

def roulette_wheel_selection(probabilities: List[float]) -> int:
    r = random.random()
    cumulative = 0.0
    for i, p in enumerate(probabilities):
        cumulative += p
        if r <= cumulative:
            return i
    return len(probabilities) - 1  # fallback in case of floating rounding

def ant_colony_optimization(
    cities: List[Tuple[float, float]],
    alpha: float = 1.0,
    beta: float = 5.0,
    evaporation_rate: float = 0.5,
    Q: float = 100.0,
    max_iterations: int = 100,
    num_ants: int = 10,
    tau0: float = 0.1,
    seed: Optional[int] = None
) -> Tuple[List[int], float]:
    if seed is not None:
        random.seed(seed)

    num_cities = len(cities)
    pheromone = initialize_pheromone(num_cities, tau0=tau0)

    best_tour: List[int] = []
    best_length = float('inf')

    for iteration in range(max_iterations):
        all_tours: List[List[int]] = []
        all_lengths: List[float] = []

        for _ in range(num_ants):
            start_city = random.randint(0, num_cities - 1)
            visited = {start_city}
            tour = [start_city]
            current_city = start_city

            while len(visited) < num_cities:
                unvisited_cities = [city for city in range(num_cities) if city not in visited]

                # compute denominator
                denominator = 0.0
                heuristics = {}
                for j in unvisited_cities:
                    dist = distance(cities[current_city], cities[j])
                    eta = 1.0 / dist if dist > 0 else 1e10
                    heuristics[j] = eta
                    denominator += (pheromone[current_city][j] ** alpha) * (eta ** beta)

                # if denominator is zero (very unlikely), use uniform probabilities
                probabilities: List[float] = []
                if denominator == 0.0:
                    probabilities = [1.0 / len(unvisited_cities)] * len(unvisited_cities)
                else:
                    for j in unvisited_cities:
                        numerator = (pheromone[current_city][j] ** alpha) * (heuristics[j] ** beta)
                        probabilities.append(numerator / denominator)

                next_city_index = roulette_wheel_selection(probabilities)
                next_city = unvisited_cities[next_city_index]
                tour.append(next_city)
                visited.add(next_city)
                current_city = next_city

            # Complete the tour by returning to start city
            tour.append(start_city)

            # Calculate total tour length
            length = 0.0
            for i in range(len(tour) - 1):
                length += distance(cities[tour[i]], cities[tour[i + 1]])

            all_tours.append(tour)
            all_lengths.append(length)

            if length < best_length:
                best_length = length
                best_tour = tour.copy()

        # Pheromone evaporation
        for i in range(num_cities):
            for j in range(num_cities):
                pheromone[i][j] *= (1.0 - evaporation_rate)
                # optional: keep pheromone non-negative
                if pheromone[i][j] < 1e-12:
                    pheromone[i][j] = 1e-12

        # Pheromone update (deposit)
        for tour, length in zip(all_tours, all_lengths):
            if length <= 0:
                continue
            deposit = Q / length
            for i in range(len(tour) - 1):
                a, b = tour[i], tour[i + 1]
                pheromone[a][b] += deposit
                pheromone[b][a] += deposit  # symmetric

        print(f"Iteration {iteration+1}/{max_iterations}, best length: {best_length:.4f}")

    return best_tour, best_length

# Example usage:
if __name__ == "__main__":
    # Define some cities (x, y)
    cities = [(0, 0), (1, 5), (5, 2), (6, 6), (8, 3)]
    best_tour, best_length = ant_colony_optimization(cities, max_iterations=50, num_ants=20, seed=42)
    print("Best tour:", best_tour)
    print("Best tour length:", best_length)


Iteration 1/50, best length: 22.3510
Iteration 2/50, best length: 22.3510
Iteration 3/50, best length: 22.3510
Iteration 4/50, best length: 22.3510
Iteration 5/50, best length: 22.3510
Iteration 6/50, best length: 22.3510
Iteration 7/50, best length: 22.3510
Iteration 8/50, best length: 22.3510
Iteration 9/50, best length: 22.3510
Iteration 10/50, best length: 22.3510
Iteration 11/50, best length: 22.3510
Iteration 12/50, best length: 22.3510
Iteration 13/50, best length: 22.3510
Iteration 14/50, best length: 22.3510
Iteration 15/50, best length: 22.3510
Iteration 16/50, best length: 22.3510
Iteration 17/50, best length: 22.3510
Iteration 18/50, best length: 22.3510
Iteration 19/50, best length: 22.3510
Iteration 20/50, best length: 22.3510
Iteration 21/50, best length: 22.3510
Iteration 22/50, best length: 22.3510
Iteration 23/50, best length: 22.3510
Iteration 24/50, best length: 22.3510
Iteration 25/50, best length: 22.3510
Iteration 26/50, best length: 22.3510
Iteration 27/50, best